# All Packages Imports

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from Bio import SeqIO
import pprint
import re
import numpy as np
import pandas as pd
from tabulate import tabulate
from urllib.request import urlopen
from bs4 import BeautifulSoup
import Bio.SwissProt as sp
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

# Retrieve all Sequences for the searched term

In [15]:
#Aceder ao NCBI
Entrez.email = 'pg45963@uminho.pt'
handle = Entrez.esearch(db = "gene", term = "staphylococcus aureus prophage", rettype="gb", retmode = "text", retmax = 4)
record = Entrez.read(handle)
print(record)

{'Count': '461', 'RetMax': '4', 'RetStart': '0', 'IdList': ['1263011', '1262987', '1262982', '1262976'], 'TranslationSet': [{'From': 'staphylococcus aureus', 'To': '"Staphylococcus aureus"[Organism] OR staphylococcus aureus[All Fields]'}], 'TranslationStack': [{'Term': '"Staphylococcus aureus"[Organism]', 'Field': 'Organism', 'Count': '147321', 'Explode': 'Y'}, {'Term': 'staphylococcus aureus[All Fields]', 'Field': 'All Fields', 'Count': '166588', 'Explode': 'N'}, 'OR', 'GROUP', {'Term': 'prophage[All Fields]', 'Field': 'All Fields', 'Count': '27307', 'Explode': 'N'}, 'AND', 'GROUP'], 'QueryTranslation': '("Staphylococcus aureus"[Organism] OR staphylococcus aureus[All Fields]) AND prophage[All Fields]'}


# Get All Sequences

In [ ]:
data = []
prot_seqs = ""
nucl_seqs = ""

for id in record["IdList"]:
    try:
        h1 = Entrez.esummary(db="gene", id=id)
        r = Entrez.read(h1)

        info = r["DocumentSummarySet"]["DocumentSummary"]
        name = info[0]["Name"]
        seqtype = info[0]["GeneticSource"]
        new_id = info[0]["GenomicInfo"][0]["ChrAccVer"]
        h1.close()

        h2 = Entrez.efetch(db="nucleotide",
                        id=new_id,
                        rettype="gb",
                        retmode="text")
        
        whole_sequence = SeqIO.read(h2, "gb")
        sequence = whole_sequence.seq
        DNAId = whole_sequence.id
        
        for value in whole_sequence.features:
            if value.type == "CDS" and value.qualifiers["locus_tag"][0] == name:
                start = value.location.start
                end = value.location.end
                dna_seq = sequence[start:end]
                protein_id = value.qualifiers["protein_id"][0]
                protein_seq = value.qualifiers["translation"][0]
                prot_desc = value.qualifiers["product"][0]

        nucl_seqs += f">{name}|{DNAId}\n{dna_seq}\n\n"
        prot_seqs += f">{name}|{protein_id}\n{protein_seq}\n\n"
        data.append([name, DNAId, f"{dna_seq[:20]}...", protein_id, f"{protein_seq[:20]}...", prot_desc])
    
    except:
        pass

f = open("1_ProteinSeqs_code.txt", "w")
g = open("1_DNASeqs_code.txt", "w")
g.write(nucl_seqs)
f.write(prot_seqs)
f.close()
g.close()

In [3]:
info_firstline = ["Name", "DNA ID", "Sequence", "Protein ID", "Protein Seq", "Protein Description"]
numpy_data = np.array(data)
numpy_data_frame = numpy_data.reshape((len(data), 6))

info_genes = pd.DataFrame(data = numpy_data_frame, columns = info_firstline)
table = tabulate(info_genes, headers='keys', tablefmt='psql')

print(table)

+-----+------------------+-------------+-------------------------+----------------+-------------------------+-------------------------------------------------------------+
|     | Name             | DNA ID      | Sequence                | Protein ID     | Protein Seq             | Protein Description                                         |
|-----+------------------+-------------+-------------------------+----------------+-------------------------+-------------------------------------------------------------|
|   0 | phiPV83p45       | NC_002486.1 | ATGGCAATGTATGAAGTGAA... | NP_061633.1    | MAMYEVKKSYTDLEKGQYLK... | phi PVL ORF 8 homologue                                     |
|   1 | phiPV83p39       | NC_002486.1 | ATGGCGGGTAGACCTAAGAA... | NP_061627.1    | MAGRPKKLLSNSNKNYTKEE... | terminase small subunit P27 family                          |
|   2 | phiPV83p50       | NC_002486.1 | ATGATTGAAAAATTGAAACA... | NP_061638.1    | MIEKLKQAPRFLKLNLQHFA... | phi PVL ORF 13 homologue      

# Verify all Genomes and the proteins that do host recognition (integrases)

In [ ]:
genomes = list(set(info_genes["DNA ID"]))
prot_all = []

for g in genomes:
    #Aceder ao NCBI
    Entrez.email = 'pg45963@uminho.pt'
    handle = Entrez.efetch(db = "nucleotide", id = g, rettype = "gb", retmode = "text")
    seq_record = SeqIO.read(handle, "gb")

    #Criar um ficheiro
    filename = f"./NCBI_Genomes/{g}.gb"
    SeqIO.write(seq_record, filename, "gb")
    handle.close()

    #Aceder aos dados
    f = open(filename)
    info = SeqIO.read(f, "gb")

    # Verifica quais são referentes a tail proteins -> proteínas que integram o genoma dentro do hospedeiro
    for value in info.features:
        try:
            if "tail" in value.qualifiers["product"][0]:
                data = [g, value.qualifiers["protein_id"][0], value.qualifiers["locus_tag"][0], value.qualifiers["product"][0]]
                prot_all.insert(0, data)
            elif "baseplate" in value.qualifiers["product"][0]:
                data = [g, value.qualifiers["protein_id"][0], value.qualifiers["locus_tag"][0], value.qualifiers["product"][0]]
                prot_all.append(data)
        except:
            pass

pprint.pprint(prot_all)

# New Table with the Proteins

In [5]:
info_firstline = ["DNA ID", "Protein ID", "Protein Tag", "Description"]
numpy_data = np.array(prot_all)
numpy_data_frame = numpy_data.reshape((len(prot_all), 4))

info_proteins = pd.DataFrame(data = numpy_data_frame, columns = info_firstline)
table = tabulate(info_proteins, headers='keys', tablefmt='psql')

print(table)

+-----+-------------+----------------+------------------+--------------------------------------------+
|     | DNA ID      | Protein ID     | Protein Tag      | Description                                |
|-----+-------------+----------------+------------------+--------------------------------------------|
|   0 | NC_024391.1 | YP_009045047.1 | HL47_gp54        | tail protein                               |
|   1 | NC_024391.1 | YP_009045046.1 | HL47_gp53        | tail family protein                        |
|   2 | NC_024391.1 | YP_009045044.1 | HL47_gp51        | tail assembly protein                      |
|   3 | NC_024391.1 | YP_009045043.1 | HL47_gp50        | tail assembly protein                      |
|   4 | NC_024391.1 | YP_009045042.1 | HL47_gp49        | tail protein                               |
|   5 | NC_024391.1 | YP_009045038.1 | HL47_gp45        | head-tail adapter protein                  |
|   6 | NC_007050.1 | YP_239738.1    | ST85ORF003       | tail protein   

# Get the Proteins from UniProt and understand their function

In [6]:
def get_dna_seq(new_id: int, name: str) -> str:
    '''Obtain the DNA sequence of the ID and name provided 

    :param new_id: DNA ID
    :type new_id: int
    :param name: Gene name to find in the GenBank
    :type name: str
    :return: Gene sequence
    :rtype: str
    '''
    h2 = Entrez.efetch(db="nucleotide",
                        id=new_id,
                        rettype="gb",
                        retmode="text")
        
    whole_sequence = SeqIO.read(h2, "gb")
    sequence = whole_sequence.seq
        
    for value in whole_sequence.features:
        if value.type == "CDS" and value.qualifiers["locus_tag"][0] == name:
            start = value.location.start
            end = value.location.end
            dna_seq = sequence[start:end]

    return dna_seq

In [7]:
tail_fibers = []
tail_fibers_protseqs = ""
tail_fibers_dnaseqs = ""

for i, id in enumerate(info_proteins["Protein ID"]):
    # Obtain entrys for uniprot details
    entrys = urlopen(f"https://www.uniprot.org/uniprot/?query={id}&columns=id&format=tab")
    soup = BeautifulSoup(entrys, 'lxml')
    text = list(soup.get_text().split())

    # Try to get the entry, if not possible means that UniProt does not have the protein in the db
    try:
        swiss_entry = text[1]

        # Obtain the info from uniprot page and write into a file
        data_swiss = urlopen(f"https://www.uniprot.org/uniprot/{swiss_entry}.txt")
        soup2 = BeautifulSoup(data_swiss, 'lxml')
        data = soup2.get_text()
        filename = f"./Uniprot_pages/{id}|{swiss_entry}.txt"
        f = open(filename, "w")
        f.write(data)
        f.close()
        
        # Open the file and analyze the information
        with open(filename) as handle:
            records = sp.parse(handle)
            for record in records:
                desc = record.description.split('=')[1].split('{')[0]
                keywords = record.keywords
                keywords.insert(0, info_proteins["Description"][i])
                seq_prot = record.sequence

        strkeywords = ", ".join(i.lower() for i in keywords)
        dna_id = info_proteins["DNA ID"][i]
        dna_seq = get_dna_seq(dna_id, info_proteins["Protein Tag"][i])

        if "fiber" in desc or "fiber" in strkeywords:
            tail_fibers.append([id, dna_id, desc, strkeywords])
            tail_fibers_protseqs += f">{id}|{desc}\n{seq_prot}\n\n"
            tail_fibers_dnaseqs += f">{dna_id}|{desc}\n{dna_seq}\n\n"
    
    except:
        pass

# Make pandas DataFrame and write content to Excel
info_firstline = ["Protein ID", "DNA ID", "Description", "Keywords"]
numpy_data = np.array(tail_fibers)
numpy_data_frame = numpy_data.reshape((len(tail_fibers), 4))

df_tail_proteins = pd.DataFrame(data = numpy_data_frame, columns = info_firstline)
table = tabulate(tail_fibers, headers='keys', tablefmt='psql')
print(table)

FB_exc_Est= pd.ExcelWriter('Tail_Fiber_Proteins.xlsx', engine='xlsxwriter') 
df_tail_proteins.to_excel(FB_exc_Est, sheet_name='Folha 1', index=False)
FB_exc_Est.save()

# Escrever as sequências
f = open("2_TailFiber_ProtSeqs.txt", "w")
g = open("2_TailFiber_DNASeqs.txt", "w")
f.write(tail_fibers_protseqs)
g.write(tail_fibers_dnaseqs)
f.close()
g.close()

+----------------+-------------+--------------------------+-----------------------------------------------------------------------------------------------------+
| 0              | 1           | 2                        | 3                                                                                                   |
|----------------+-------------+--------------------------+-----------------------------------------------------------------------------------------------------|
| YP_009204432.1 | NC_028864.1 | Tail fiber               | tail fiber, coiled coil, reference proteome                                                         |
| NP_803346.1    | NC_004616.1 | Tail fiber protein       | tail tape measure protein, coiled coil, reference proteome                                          |
| YP_003857091.1 | NC_014460.1 | Tail fiber               | tail fiber, coiled coil, reference proteome                                                         |
| YP_009209036.1 | NC_028913

In [19]:
def prot_desc(new_id: int) -> str:
    '''Obtain the protein description of the ID provided

    :param new_id: Gene ID
    :type new_id: int
    :return: Protein description of the gene
    :rtype: str
    '''
    Entrez.email = 'pg45963@uminho.pt'

    h1 = Entrez.esummary(db="gene", id=new_id)
    r = Entrez.read(h1)

    info = r["DocumentSummarySet"]["DocumentSummary"]
    name = info[0]["Name"]

    h2 = Entrez.efetch(db="nucleotide",
                            id=new_id,
                            rettype="gb",
                            retmode="text")
            
    whole_sequence = SeqIO.read(h2, "gb")
            
    for value in whole_sequence.features:
        if value.type == "CDS" and value.qualifiers["locus_tag"][0] == name:
            prot_desc = value.qualifiers["product"][0]
    
    return prot_desc

# Blasts to obtain more tail fiber proteins -> Taking too much time

In [ ]:
hits = []
ids = []

with open("2_TailFiber_DNASeqs.txt", "r") as f:
    lines = f.readlines()
    seqs = []
    for l in lines:
        if l[0:3] == "ATG": seqs.append(l)

    all_DNA_Seqs_WBlast = ''.join(lines)
    for seq in seqs:
        handle = NCBIWWW.qblast("blastn", "nt", seq.format('fasta'), hitlist_size=30)
        blast_records = NCBIXML.parse(handle)

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                organism = alignment.hit_def[alignment.hit_def.find("[")+1 : alignment.hit_def.find("]")]
                for hsp in alignment.hsps:
                    identifier = alignment.hit_id.split("|")[3]
                    if identifier not in ids:
                        info_hit = (identifier, alignment.length, hsp.expect, hsp.score, organism)
                        hits.append(info_hit)
                        ids.append(identifier)

                        all_DNA_Seqs_WBlast += f">{identifier}\n{hsp.sbjct}\n\n"

        pprint.pprint(hits)
                    

info_firstline = ["Accession Number", "Lenght", "E-Value", "Score", "Organism"]
numpy_data = np.array(hits)
numpy_data_frame = numpy_data.reshape((len(numpy_data), 5))

info_blast = pd.DataFrame(data = numpy_data_frame, columns = info_firstline)
print(tabulate(info_blast, headers='keys', tablefmt='psql'))

f = open("All_DNA_Seqs_WBlast.txt", "w")
f.write(all_DNA_Seqs_WBlast)
f.close()

# Remover Duplicados do Blast e inverter Seqs

In [5]:
def comp_inverse(seq):
    change = {"A": "T", "T": "A", "G": "C", "C": "G"}
    new = "".join(change[i] for i in seq)
    return new[::-1]

with open("All_DNA_Seqs_WBlast.txt", "r") as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if "|" in lines[i]: lines[i] = f"{lines[i].split('|')[0]}\n"
    
    identifiers = {}
    for i in lines:
        if i not in identifiers and i[0] == ">": identifiers[i] = 0

    size = len(max(lines, key = len))               # Para aumentar os espaços e terem o mesmo tamanho
    string = ""
    pos = 0
    while pos < len(lines) - 1:
        if lines[pos] in identifiers and identifiers[lines[pos]] == 1:
            pos += 3
        elif lines[pos] in identifiers and identifiers[lines[pos]] == 0:
            string += lines[pos]
            identifiers[lines[pos]] += 1
            pos += 1
        else:
            space = "".join("." for _ in range(size - len(lines[pos])))
            if lines[pos][-4:-1] == "CAT": seq = comp_inverse(lines[pos][:-1])                  # Trocar seqs que estejam invertidas e reversas
            else: seq = lines[pos][:-1]
            string += f"{seq}{space}\n\n"
            pos += 2

    f = open("DNA_Seqs_Final.txt", "w")
    f.write(string)
    f.close

# Alinhamentos

In [181]:
def get_prot(seq: str) -> str:
    '''Obtain the protein throw the DNA sequence

    :param seq: DNA sequence
    :type seq: str
    :return: protein sequence
    :rtype: str
    '''
    if seq.find("-"):
        seq = seq[:seq.find("-")]

    amino_codons = {"F":["TTT", "TTC"], "L":["TTA", "TTG", "CTT", "CTC", "CTA", "CTG"], "I":["ATT", "ATC", "ATA"], "M":"ATG", "V":["GTT", "GTC", "GTA", "GTG"], "S":["TCT", "TCC", "TCA", "TCG", "AGT", "AGC"], "P":["CCT", "CCC", "CCA", "CCG"], "T":["ACT", "ACC", "ACA", "ACG"], "A":["GCT", "GCC", "GCA", "GCG"], "Y":["TAT", "TAC"], "_":["TAA", "TAG", "TGA"], "H":["CAT", "CAC"], "Q":["CAA", "CAG"], "N":["AAT", "AAC"], "K":["AAA", "AAG"], "D":["GAT", "GAC"], "E":["GAA", "GAG"], "C":["TGT", "TGC"], "W":"TGG", "R":["CGT", "CGC", "CGA", "CGG", "AGA", "AGG"], "G":["GGT", "GGC", "GGA", "GGG"]}
    aa = ""
    add = False
    codons = re.findall("(...)", str(seq))
    for codon in codons:
        for amino, codoes in amino_codons.items():
            if codon == "ATG" and add is False:
                aa += "M"
                add = True
            elif codon in codoes and add:
                aa += amino
                if amino == "_": return aa
    return aa

def get_codons(seq: str) -> str:
    '''Obtain the sequence divided by codons

    :param seq: DNA sequence
    :type seq: str
    :return: DNA sequence dividid by codons
    :rtype: str
    '''
    res = ""
    count = 0
    for ele in seq:
        if ele != "-": count += 1
        if count % 3 == 0 and ele != "-": res += f"{ele} "
        else: res += ele
    return res

## Realiza-se um alinhamento global para dividir todas as sequências por níveis de valor relativamente a um consensus geral

In [184]:
alignments = AlignIO.read("DNA_Seqs_Final.txt", "fasta")
summary_align = AlignInfo.SummaryInfo(alignments)

cons = Seq(summary_align.dumb_consensus())

to_write = ""
values = {}

for alignment in alignments:
    to_write += f">{alignment.id}\n\n"
    seq = Seq(alignment.seq)
    alignments_prot = pairwise2.align.globalxx(seq, cons)

    for align in alignments_prot: 
        to_write += format_alignment(*align)
        value = int(format_alignment(*align).split("=")[1])
        not_inserted = True
        for v in values.keys():
            if value in v and not_inserted: 
                values[v].append((alignment.id, seq))
                not_inserted = False
        
        if not_inserted: values[range(max(value - 100, 0), max(value + 100, 0))] = [(alignment.id, seq)]
        to_write += "\n\n\n"
        break

f = open("./Alignments/DNA_alignment_v01.txt", "w")
f.write(to_write)
f.close()

## Consensus de cada divisão de sequências para escrita do alinhamento de cada

In [ ]:
print(values.keys())
to_write = ""
for value, info in values.items():
    with open("temp.txt", "w") as f:
        size = len(max([tup[1] for tup in info], key = len))
        for ident, seq in info:
            space = "".join("." for _ in range(size - len(seq)))
            f.write(f">{ident}\n{seq}{space}\n\n")
    
    to_write += f"------------ {value} ------------\n\n"
    alignments = AlignIO.read("temp.txt", "fasta")
    summary_align = AlignInfo.SummaryInfo(alignments)

    cons = Seq(summary_align.dumb_consensus())

    for alignment in alignments:
        to_write += f">{alignment.id}\n\n"
        seq = Seq(alignment.seq)
        end = seq.find('.')
        alignments_prot = pairwise2.align.globalxx(seq[:end], cons[:end])

        for align in alignments_prot:
            seqA = get_codons(align.seqA)
            seqB = get_codons(align.seqB)
            to_write += format_alignment(seqA, seqB, align.score, align.start, align.end)
            to_write += "\n\n\n"
            break

f = open("./Alignments/DNA_alignment_v02.txt", "w")
f.write(to_write) 
f.close()